In [6]:
# For plotting
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# For matrix math
import numpy as np
# Plot style
%matplotlib notebook
# Set plot size in notebook
plt.rcParams["figure.figsize"] = (5, 5)

In [7]:
# Import a Dataset used for Novelty Detection

### Source ###
# https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names
# University of Wisconsin Hospitals, Madison from Dr. William H. Wolberg
# O. L. Mangasarian and W. H. Wolberg: "Cancer diagnosis via linear programming", SIAM News, Volume 23, Number 5, September 1990, pp 1 & 18.

wbc = pd.read_csv("../2.Data/WBC_Minority.csv")
wbc.head(n=5)

,ID,Clump Thickness,Cell Size,Cell Shape,Marginal Adhesion,Epithelial,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class(2 - benign 4 - malignant)
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [8]:
# Create input data ndarray

# Remove bare nuclei column due to incomplete data
# Slice into test and training data
X = np.append(wbc.values[:,1:6],wbc.values[:,7:10], axis=1)
X_test = X[:79,:] # Polluted
X = np.delete(X,np.s_[:79],0)
# Slice labels into test and training data
y = wbc.values[:,10]
y_test = y[:79] # Polluted
y = np.delete(y,np.s_[:79])
print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)
print(X[:5,:])

(400, 8)
(79, 8)
(400,)
(79,)
[[1 3 1 2 2 5 3 2]
 [3 3 2 1 2 3 1 1]
 [1 1 1 1 2 1 1 1]
 [8 3 3 1 2 3 2 1]
 [1 1 1 1 4 1 1 1]]


In [9]:
# Calculate distance matrix
instances = X.shape[0]
distance = np.zeros([instances, instances])
print(distance.shape)

for i in range(instances):
    for j in range(instances):
        distance[i,j] = np.linalg.norm(X[i,:] - X[j,:])
        
print(distance[10:15,10:15])

(400, 400)
[[0.         9.21954446 8.30662386 8.36660027 7.34846923]
 [9.21954446 0.         3.74165739 3.         3.        ]
 [8.30662386 3.74165739 0.         1.73205081 1.73205081]
 [8.36660027 3.         1.73205081 0.         1.41421356]
 [7.34846923 3.         1.73205081 1.41421356 0.        ]]


In [10]:
# Calculate k-distance for each point (distance to k-th nearest neighbor)
k = 50 
k_dist = np.zeros([instances])

for i in range(instances):
    k_dist[i] = np.sort(distance[i,:])[k - 1]

print(np.sort(distance[0,:]).shape)
print(k_dist[10:15])

(400,)
[8.         2.23606798 1.41421356 1.41421356 1.41421356]


In [11]:
# Assign k-distance neighborhoods
# neighbors[i,j] = 1 iff point j is in the k-distance neighborhood of i
# non-symmetrical!

N_k = np.zeros([instances, instances])

for i in range(instances):
    for j in range(instances):
        if distance[i,j] <= k_dist[i]:
            N_k[i,j] = 1 
            
# A point is not the neighbor of itself:
neighbors = N_k - np.eye(instances)

print(np.eye(instances))
print(neighbors.shape)
print(neighbors[10:15,10:15])

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(400, 400)
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]]


In [12]:
# Calculate reachability distance for each point from each point
# Motivation for this definition is numeric stability

# Copy distance matrix
r_dist = distance

# Replace relevant entries for reachability-distance
for i in range(instances):
    for j in range(instances):
        if i != j and r_dist[i,j] < k_dist[i]:
            r_dist[i,j] = k_dist[i]
            
# Look at results
print(r_dist[10:15,10:15])

[[0.         9.21954446 8.30662386 8.36660027 8.        ]
 [9.21954446 0.         3.74165739 3.         3.        ]
 [8.30662386 3.74165739 0.         1.73205081 1.73205081]
 [8.36660027 3.         1.73205081 0.         1.41421356]
 [7.34846923 3.         1.73205081 1.41421356 0.        ]]


In [13]:
# Calculate local reachability density
lrd = np.zeros(instances)

for i in range(instances):
    lrd[i] = np.sum(neighbors[i,:]) / np.dot(r_dist[i,:],neighbors[i,:])
    
print(lrd[10:15])

### Check for zero entries ###  
for i in range(instances):
    if np.dot(r_dist[i,:],neighbors[i,:]) == 0:
        print(np.dot(r_dist[i,:],neighbors[i,:]))

[0.125      0.4472136  0.70710678 0.70710678 0.70710678]


In [14]:
# Calculate LOFs
lof = np.zeros(instances)

for i in range(instances):
    lof[i] = (np.dot(lrd,neighbors[i,:])/(np.sum(neighbors[i,:]))) / lrd[i]
                       
print(lof[10:15])
print(lof.shape)

[6.46138677 1.47171121 1.27182765 1.23842551 1.2787892 ]
(400,)


In [15]:
# We now have LOF Data for every training instance
# Use it to classify test samples

def LOF(new_point):
    # Calculate distance to every other point
    position = np.empty(instances)
    for i in range(instances):
        position[i] = np.linalg.norm(new_point-X[i,:])

    # Calculate k-distance
    k_dist_new = np.sort(position)[k - 1]

    # Define Neighborhood
    N_new = np.zeros(instances)
    for i in range(instances):
        if position[i] <= k_dist_new:
                N_new[i] = 1 

    # Define reachability distance
    r_new = position
    for i in range(instances):
        if r_new[i] < k_dist_new:
            r_new[i] = k_dist_new

    # With this calculate lrd(new_point)
    lrd_new = np.sum(N_new)/np.dot(r_new,N_new)

    # Use known np.dot(lof(test), Neighborhood) to calculate lof(new_point)
    lof_new = (np.dot(N_new,lrd) / lrd_new) / np.sum(N_new)
    return(lof_new)

In [16]:
# Calculate LOF values for test set
lof_values = np.empty(X_test.shape[0])
for t in range(X_test.shape[0]):
    lof_values[t] = LOF(X_test[t])

In [17]:
# Assess results
cutoff = 2.25

# Generate confusion matrix:
tp = 0
fn = 0
tn = 0
fp = 0
results = np.empty((X_test.shape[0],1),dtype='object')

for i in range(X_test.shape[0]):
    pdfvalue = lof_values[i]
    
    if np.isin(i,np.where(y_test==2)) and pdfvalue < cutoff:
        tp = tp + 1
        results[i] = "True Positive"
    if np.isin(i,np.where(y_test==2)) and pdfvalue >= cutoff:
        fn = fn + 1
        results[i] = "False Negative"
    if np.isin(i,np.where(y_test==4)) and pdfvalue >= cutoff:
        tn = tn + 1
        results[i] = "True Negative"
    if np.isin(i,np.where(y_test==4)) and pdfvalue < cutoff:
        fp = fp + 1
        results[i] = "False Positive"

print("Confusion Matrix:\n",tp,fp,"\n",fn,tn)

Confusion Matrix:
 50 2 
 8 19


In [18]:
# F1 Score
f1 = (2*tp)/(2*tp+fp+fn)
print(f1)

0.9090909090909091
